In [ ]:
# 如果資料是 dict 那就要用 BaseModel
# 如果資料是 list 那就要用 root
#pydantic 就是專門讀取json資料

In [40]:
from dotenv import load_dotenv
import requests
load_dotenv()

def get_youbike_str()->str:
    url = f'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    try:
        r = requests.get(url)
        r.raise_for_status()   #這樣就不用檢查 200
    except Exception as e:
        print(f'發生錯誤: {e}')

    return r.text


youbike_data:str = get_youbike_str()

In [41]:
from pydantic import RootModel,BaseModel,Field,field_serializer,field_validator
from datetime import datetime
class Taipei_youbike(BaseModel):
    編號:str = Field(alias='sno')
    站點名稱:str = Field(alias='sna')
    站點區域:str = Field(alias='sarea')
    站點時間:datetime = Field(alias='mday')
    站點位址:str = Field(alias='ar')
    是否開放:bool = Field(alias='act')
    車輛總數:int = Field(alias='total')
    可藉車輛:int = Field(alias='available_rent_bikes')
    可還車輛:int = Field(alias='available_return_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')

    @field_validator('站點名稱')
    @classmethod
    def abbr(cls,name:str)->str:
        return name.split('_')[1]
    
    @field_serializer('緯度','經度')
    def four_digits(self,value:float)->float:
        return round(value,ndigits=4)
    
    @field_serializer('站點時間')
    def datetime_to_str(self,date:datetime)->str:
        return date.strftime('%Y-%m-%d %H:%M:00')
    

class Root_Youbike(RootModel):
    root: list[Taipei_youbike]

    



model = Root_Youbike.model_validate_json(youbike_data)


In [42]:
#使用model_dump 轉成python資料
youbike_data:list[dict] = model.model_dump()
for youbike in youbike_data:
    print(youbike)

{'編號': '500101001', '站點名稱': '捷運科技大樓站', '站點區域': '大安區', '站點時間': '2024-11-25 16:30:00', '站點位址': '復興南路二段235號前', '是否開放': True, '車輛總數': 28, '可藉車輛': 6, '可還車輛': 21, '緯度': 25.0261, '經度': 121.5436}
{'編號': '500101002', '站點名稱': '復興南路二段273號前', '站點區域': '大安區', '站點時間': '2024-11-25 16:31:00', '站點位址': '復興南路二段273號西側', '是否開放': True, '車輛總數': 21, '可藉車輛': 9, '可還車輛': 12, '緯度': 25.0256, '經度': 121.5436}
{'編號': '500101003', '站點名稱': '國北教大實小東側門', '站點區域': '大安區', '站點時間': '2024-11-25 16:28:00', '站點位址': '和平東路二段96巷7號', '是否開放': True, '車輛總數': 16, '可藉車輛': 8, '可還車輛': 8, '緯度': 25.0243, '經度': 121.5412}
{'編號': '500101004', '站點名稱': '和平公園東側', '站點區域': '大安區', '站點時間': '2024-11-25 16:31:00', '站點位址': '和平東路二段118巷33號', '是否開放': True, '車輛總數': 11, '可藉車輛': 4, '可還車輛': 7, '緯度': 25.0235, '經度': 121.5428}
{'編號': '500101005', '站點名稱': '辛亥復興路口西北側', '站點區域': '大安區', '站點時間': '2024-11-25 16:31:00', '站點位址': '復興南路二段368號', '是否開放': True, '車輛總數': 16, '可藉車輛': 2, '可還車輛': 13, '緯度': 25.0215, '經度': 121.543}
{'編號': '500101006', '站點名稱': '復興南路二段280號前', '站點區域': '大安